Iterative Solvers
===

So far we have used direct solvers to solve the linear system of equations. Although a direct solver can profit from the sparse matrix, it's arithmetic complexity is sub-optimal. For large-scale problems iterative solvers are a must.

The conjugate gradient (cg) method is the standard method for symmetric and positive definite matrices. It's convergence rate depends on a preconditioner, what is a cheap approximative inverse to the matrix.

In [1]:
import netgen.gui
from ngsolve import *
%gui tk

We generate a 3D geometry and mesh using the constructive solid geometry (CSG) modeler of Netgen:

In [19]:
from netgen.csg import *
geo = CSGeometry()
cube = OrthoBrick (Pnt(0,0,0), Pnt(1,1,1)).bc("outer")
cyl = Cylinder( Pnt(1,0.5,0.5), Pnt(0,0.5,0.5), 0.2).bc("inner")
geo.Add (cube-cyl, col=(0,0,1))
geo.Draw()

mesh = Mesh(geo.GenerateMesh(maxh=0.1))
mesh.Refine()
mesh.Refine()
mesh.Curve(3)
Draw (mesh)

In [20]:
fes = H1(mesh, order=3, dirichlet="outer", flags = { "wb_withoutedges" : True })
print ("we have", fes.ndof, "unknowns")
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))

f = LinearForm(fes)
f += SymbolicLFI(v)

# c = Preconditioner(a, "local")
# c = Preconditioner(a, "direct", inverse="sparsecholesky")
c = Preconditioner(a, "bddc")

gfu = GridFunction(fes)

we have 1560464 unknowns


assemble system and setup preconditioner in parallel:

In [21]:
ngsglobals.msg_level=5
with TaskManager():
    a.Assemble()
    f.Assemble()

solve the system using the preconditioned conjugate gradient method:

In [ ]:
from ngsolve.solvers import CG

with TaskManager():
    CG (mat=a.mat, pre=c.mat, rhs=f.vec, sol=gfu.vec, 
        printrates=True, maxsteps=200)
Draw(gfu)

it =  0  err =  0.28712089056269857
it =  1  err =  0.10749286926619048
it =  2  err =  0.08752453658398944
it =  3  err =  0.07039491341804945
it =  4  err =  0.05077599365114595
it =  5  err =  0.03360351715919447
it =  6  err =  0.02170035970419196
it =  7  err =  0.013913972568441249
it =  8  err =  0.00941766995942612
it =  9  err =  0.006567387116831617
it =  10  err =  0.004427831463017225
it =  11  err =  0.0029178418259392124
it =  12  err =  0.0019173542581778222
it =  13  err =  0.0012463460302125566
it =  14  err =  0.0008156179258766561